In [ ]:
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:90% !important; }</style>")) 

# Praca z JSON API

Czym jest API? 
- API to skrót od Application Programming Interface
- w zależności od konkretnego zastosowania może oznaczać trochę coś innego
- w naszym przypadku API jest pewną usługą webową, która pozwala pozyskać dane w ustrukturyzowanej postaci z bazy danych do której nie mamy bezpośredniego dostępu
- API to rodzaj pośrednika (interface) między bazą danych a naszą aplikacją

Jak działa API?
- możemy łączyć się z nim poprzez przeglądarkę lub wewnątrz naszego skryptu za pomocą biblioteki `requests`
- wysyłamy zapytanie typu `GET` a w odpowiedzi otrzymujemy dane w formacie JSON (czasem może to być również XML, ale tym nie będziemy się zajmować)

Czym jest JSON
- JavaScript Object Notation
- ustrukturyzowana forma zapisu danych
- JSON przypomina słownik w Pythonie. Dane przechowywane są na zasadzie klucz-wartość

###### Przykład danych w formacie JSON

In [ ]:
{"employees": [
                {"id": 14235, "name": "Bob", "date_of_birth": "1982-04-15", "department": "Sales"},
                {"id": 28134, "name": "Susan", "date_of_birth": "1991-09-01", "department": "IT"},
#                 ...
                {"id": 89231, "name": "Andrew", "date_of_birth": "1990-02-15", "department": "Sales"}
              ],
 "departments": ["Sales", "IT", "HR"]}

## currencylayer API

Można znaleźć wiele darmowych API, które udostępniają dane dotyczące finansów, np. ceny metali szlachetnych czy kursy walut. Jako pierwszy przykład zobaczmy jak działa API currencylayer.

Szczegółową dokumentację można znaleźć tutaj -> https://currencylayer.com/documentation . Zamiast tego przejdźmy od razu do przykładu praktycznego żeby zobaczyć o co właściwie chodzi.

### Wywołanie API

Przykładowe wywołanie API wygląda w następujacy sposób:

http://api.currencylayer.com/historical?access_key=bef18344d09a9963fda9d0c8402ace0e&date=2020-12-12&currencies=EUR,PLN&format=1

Otwórzmy powyższy link żeby sprawdzić co się stanie.

Uwaga: 
1. Walutą odniesienia jest dolar amerykański. W darmowej wersji API ten parametr jest niemodyfikowalny
2. API zwykle mają limity requestów dla danego klucza. Szczegółowe informacje na ten temat powinny znajdować się w dokumentacji. W przypadku currencylayer jest to 250 wywołań na miesiąc (w planie darmowym)

### Struktura adresu URL

Rozłóżmy URL na czynniki pierwsze:

http:// api.currencylayer.com/ historical?access_key= API_KEY &date= DATE &currencies= CURRENCIES_LIST &format=1

Nasze zapytanie precyzujemy podając wymagane argumenty. Informacje jak to zrobić możemy znaleźc w dokumentacji API. W tym przypadku potrzebujemy:
- klucz API
- datę
- listę walut jakie nas interesują

### Customizacja adresu URL

In [3]:
API_KEY = # "bef18344d09a9963fda9d0c8402ace0e"  # wpisz swój klucz API
DATE = "2019-05-12"
CURRENCIES_LIST = "EUR,PLN"


my_url = f"http://api.currencylayer.com/historical?access_key={API_KEY}&date={DATE}&currencies={CURRENCIES_LIST}&format=1"
my_url

SyntaxError: invalid syntax (<ipython-input-3-934ecd882314>, line 1)

### W jaki sposób pobrać dane?

In [2]:
import requests

response = requests.get(my_url)
response_json = response.json()

response_json

NameError: name 'my_url' is not defined

In [ ]:
usd_per_pln = response_json["quotes"]["USDPLN"]
usd_per_eur = response_json["quotes"]["USDEUR"]

print(usd_per_pln)
print(usd_per_eur)

Zanim przejdziemy dalej zapraszam do samodzielnej zabawy z API i własnej eksploracji jego możliwości.

## OpenWeatherMap API
### Prognoza pogody na 5 dni z 3h dokładnością

In [8]:
CITY = "Krakow"
API_KEY = "7d0c48134ae346811fa50cf99109251f"  # wpisz swój klucz API

my_url = f"http://api.openweathermap.org/data/2.5/forecast?q={CITY}&appid={API_KEY}"
my_url

'http://api.openweathermap.org/data/2.5/forecast?q=Krakow&appid=7d0c48134ae346811fa50cf99109251f'

In [9]:
response = requests.get(my_url)
response_json = response.json()

response_json

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1652184000,
   'main': {'temp': 292.25,
    'feels_like': 291.31,
    'temp_min': 291.47,
    'temp_max': 292.25,
    'pressure': 1023,
    'sea_level': 1023,
    'grnd_level': 997,
    'humidity': 42,
    'temp_kf': 0.78},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 0},
   'wind': {'speed': 2.88, 'deg': 93, 'gust': 2.01},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2022-05-10 12:00:00'},
  {'dt': 1652194800,
   'main': {'temp': 291.89,
    'feels_like': 290.99,
    'temp_min': 291.18,
    'temp_max': 291.89,
    'pressure': 1022,
    'sea_level': 1022,
    'grnd_level': 995,
    'humidity': 45,
    'temp_kf': 0.71},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 0},
   'wind': {'speed': 2.88, 'deg': 91, 'gust': 2.04},
   'visibility': 10000

Przy bardziej złożonych odpowiedziach kluczowe jest zrozumienie struktury danych. Do tego przydatna jest przeglądarka, np. Firefox albo Chrome (z dodatkiem JSON Formatter)

In [10]:
dates = []
temperatures = []

for forecast in response_json["list"]:
    dates.append(forecast["dt_txt"])
    temperatures.append(forecast["main"]["temp"] - 273.15)

In [11]:
dates

['2022-05-10 12:00:00',
 '2022-05-10 15:00:00',
 '2022-05-10 18:00:00',
 '2022-05-10 21:00:00',
 '2022-05-11 00:00:00',
 '2022-05-11 03:00:00',
 '2022-05-11 06:00:00',
 '2022-05-11 09:00:00',
 '2022-05-11 12:00:00',
 '2022-05-11 15:00:00',
 '2022-05-11 18:00:00',
 '2022-05-11 21:00:00',
 '2022-05-12 00:00:00',
 '2022-05-12 03:00:00',
 '2022-05-12 06:00:00',
 '2022-05-12 09:00:00',
 '2022-05-12 12:00:00',
 '2022-05-12 15:00:00',
 '2022-05-12 18:00:00',
 '2022-05-12 21:00:00',
 '2022-05-13 00:00:00',
 '2022-05-13 03:00:00',
 '2022-05-13 06:00:00',
 '2022-05-13 09:00:00',
 '2022-05-13 12:00:00',
 '2022-05-13 15:00:00',
 '2022-05-13 18:00:00',
 '2022-05-13 21:00:00',
 '2022-05-14 00:00:00',
 '2022-05-14 03:00:00',
 '2022-05-14 06:00:00',
 '2022-05-14 09:00:00',
 '2022-05-14 12:00:00',
 '2022-05-14 15:00:00',
 '2022-05-14 18:00:00',
 '2022-05-14 21:00:00',
 '2022-05-15 00:00:00',
 '2022-05-15 03:00:00',
 '2022-05-15 06:00:00',
 '2022-05-15 09:00:00']

In [12]:
temperatures

[19.100000000000023,
 18.74000000000001,
 14.79000000000002,
 9.890000000000043,
 9.200000000000045,
 9.470000000000027,
 15.200000000000045,
 21.53000000000003,
 23.710000000000036,
 22.350000000000023,
 17.640000000000043,
 15.410000000000025,
 16.32000000000005,
 13.470000000000027,
 18.83000000000004,
 24.27000000000004,
 25.600000000000023,
 21.960000000000036,
 16.0,
 15.129999999999995,
 14.660000000000025,
 13.420000000000016,
 15.79000000000002,
 18.970000000000027,
 19.700000000000045,
 19.74000000000001,
 16.370000000000005,
 13.640000000000043,
 12.660000000000025,
 11.990000000000009,
 14.689999999999998,
 17.55000000000001,
 19.710000000000036,
 19.870000000000005,
 15.140000000000043,
 12.57000000000005,
 12.590000000000032,
 10.970000000000027,
 13.670000000000016,
 15.370000000000005]

### Poziom zanieczyszczenia powietrza
API OpenWeatherMap udostępnia również dane dotyczące zanieczyszczenia powietrza w danej chwili (bez prognozy) 

In [13]:
LATITUDE = "50"
LONGITUDE = "20"
API_KEY = "7d0c48134ae346811fa50cf99109251f"

my_url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={LATITUDE}&lon={LONGITUDE}&appid={API_KEY}"
my_url

'http://api.openweathermap.org/data/2.5/air_pollution?lat=50&lon=20&appid=7d0c48134ae346811fa50cf99109251f'

In [14]:
response = requests.get(my_url)
response_json = response.json()

In [15]:
pm2_5_level = response_json["list"][0]["components"]["pm2_5"]
pm10_level = response_json["list"][0]["components"]["pm10"]

In [16]:
print(pm2_5_level)
print(pm10_level)

3.66
4.17


### Zadanie do samodzielnego wykonania

Korzystając z OpenWeatherMap i kodu napisanego poniżej stwórz mapę zanieczyszczenia powietrza w Polsce. Wizualizacja została już zaimplementowana poniżej na przykładowych wynikach.

Czas: ok. 15 minut

In [21]:
longitudes = [20, 21, 18]
latitudes = [51, 52, 54]
city_names = ["Miasto A", "Miasto B", "Miasto C"]
pm10 = []

pm10 = [] 
API_KEY = "00fc2ef4807d24f195c8effa4a949500"

my_url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={latitudes}&lon={longitude}&cit={cities}&appid={API_KEY}"
my_url

NameError: name 'longitude' is not defined

In [ ]:
response = requests.get(my_url)
response_json = response.json()

###### Przykładowe wyniki

In [ ]:
longitudes = [20, 21, 18]
latitudes = [51, 52, 54]
cities = ["Miasto A", "Miasto B", "Miasto C"]

In [ ]:
pm10s = [20, 40, 60]

#### Mapa z naniesionymi wynikami
Po prostu uruchom poniższą komórkę.

Najedź kursorem na marker żeby zobaczyć jaki jest poziom zanieczyszczenia powietrza 

In [23]:
try:
    import folium
except ModuleNotFoundError:
    !pip install folium
    import folium

m = folium.Map(location=[52, 20], zoom_start=5)

for lon, lat, city, pm10 in zip(longitudes, latitudes, cities, pm10s):
    if pm10 > 50:
        color = "red"
    elif pm10 <= 50 and pm10 > 30:
        color = "yellow"
    else:
        color = "green"
    
    folium.CircleMarker([lat, lon], color=color, tooltip=f"{city}: pm10={pm10}ug/m3").add_to(m)
    
from branca.element import Figure
fig = Figure(width=600, height=400)
folium.Map(location=[12, 12], zoom_start=2)
fig.add_child(m)

You should consider upgrading via the '/Users/grzegorzpiwowarczyk/.pyenv/versions/3.8.4/bin/python3.8 -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'folium'

## GUS API
### Instrukcja

Główny Urząd Statystyczny również wystawia API, nawet kilka. Poznamy teraz API Bank Danych Lokalnych.

Instrukcja: https://api.stat.gov.pl/Home/BdlApi . Obsługa tego API nie jest już tak prosta jak poprzednich.

Skrót instrukcji:
1. Wychodzimy od następującego adresu URL: https://bdl.stat.gov.pl/api/v1/subjects?lang=pl&format=json 
2. Szukamy obszaru, który nas interesuje i sprawdzamy jego id. Na przykład: CENY - K15, FINANSE PRZEDSIĘBIORSTW (DANE KWARTALNE) - K43 itd...
3. Korzystamy z następującego adresu URL, https://bdl.stat.gov.pl/api/v1/subjects?parent-id=K15&format=json&lang=pl - jako parametr 'parent-id' wstawiamy id obszaru, który nas interesuje, np. K15 (ceny) 
4. Po raz kolejny wybieramy id podobszaru, który nas interesuje. W przypadku np. 'PRZECIĘTNE CENY DETALICZNE TOWARÓW I USŁUG KONSUMPCYJNYCH' jest to G188. Nasz kolejny URL wygląda tak: https://bdl.stat.gov.pl/api/v1/subjects?parent-id=G188&format=json&lang=pl (podmieniliśmy parent-id). Postępujemy w ten sposób tak długo jak parametr 'hasVariables' ma wartość False.
5. Po raz kolejny wybieramy co dokładnie nas interesuje. Załóżmy że jest to 'Żywność i napoje bezalkoholowe' (id P1466). Zauważmy, że 'hasVariables' przyjęło teraz wartość True.
6. Ponieważ hasVariables==True to bierzemy nowy wzór adresu URL (oraz ostatnie id). W naszym przypadku id to P1466, a URL to https://bdl.stat.gov.pl/api/v1/variables?subject-id=P1466&format=json&lang=pl&page-size=100 
7. Otrzymaliśmy już konkretne produkty (zmienne - variables) oraz ich numery id. 
8. Kiedy znamy już numery id produktów, które nas interesują - możemy wyszukiwać ceny tych produktów. Do tego skorzystamy z następującego adresu (na przykładzie id=4992 - ryż za 1 kg) https://bdl.stat.gov.pl/api/v1/data/by-variable/4992?format=json&unit-level=0

### Wysłanie zapytania - nagłówki

In [24]:
url = "https://bdl.stat.gov.pl/api/v1/data/by-variable/4992?format=json&unit-level=0"

response = # requests.get(url, headers={"X-ClientId":"70353ce7-9085-40fb-358e-08d8cf58ce0f"})  # wpisz własny klucz API, pamiętaj o limicie requestów

SyntaxError: invalid syntax (<ipython-input-24-1700c64a16a9>, line 3)

In [ ]:
response.json()

### Zadanie
Stwórz raport dotyczący zmiany cen towarów i/lub usług w czasie. Masz pełną dowolność tego co znajdzie się w raporcie oraz jaka będzie jego forma. Postaraj się wykorzystać jak najwięcej danych udostępnianych przez API. Możesz oprzeć się na cenie jedzenia, ale również na innych kategoriach. Możesz zwizualizować zgromadzone dane w postaci odpowiednio sformatowanego słownika (struktury typu JSON), za pomocą obiektu `DataFrame` z biblioteki pandas lub na wykresie, na przykład korzystając z biblioteki matplotlib.

Eksperymentuj. Spróbuj skorzystać z funkcji API, które nie zostały omówione, np. z pobierania danych na poziomie lokalnym a nie krajowym. Skorzystaj w tym celu z instrukcji udostępnionej przez GUS.

Uwaga: nie wszystkie produkty mają podane ceny we wszystkich latach.

Czas: ok. 60 minut

In [ ]:
# ...

## Projekt - wyznaczenie zmian temperatury na przestrzeni wielolecia

Czas na projekt końcowy, w którym ponownie użyjemy API pogodowego. Tym razem będzie to API Meteostat, które udostępnia między innymi historyczne dane pogodowe. API znajduje się pod adresem https://dev.meteostat.net/api/

Postaraj się znaleźć samodzielnie informacje dotyczące tego jak pozyskać klucz API oraz pobrać zasoby, których potrzebujesz.

Twoim zadaniem będzie policzyć średnią temperaturę w ostatnich latach w wybranej stacji pogodowej (jednej lub więcej). API udostępnia dane z wielu lat (w przypadku Warszawy historia sięga lat 70. XX wieku). Zgromadzone dane zwizualizuj na wykresie korzystając z biblioteki matplotlib -> https://matplotlib.org/2.0.2/index.html . Wykorzystaj np. funkcję plot -> https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html

Numery id poszczególnych stacji znajdziesz na stronie https://meteostat.net/en wybierając stację na interaktywnej mapie. Kliknij na stację, która Cię interesuje a następnie weź numer stacji z adresu URL strony do której Cię przeniosło.

Uwaga: czasami - na przykład jeśli przekroczysz dozwoloną liczbę requestów - odpowiedź nie zostanie zwrócona. Jeśli podejrzewasz taki błąd to sprawdź co zwraca `requests.get(...)`. Jeśli jest to `[200]` - odpowiedź została zwrócona poprawnie. Jeśli jednak kod odpowiedzi jest inny, na przykład zaczyna się od cyfry 4 lub 5, świadczy to o błędzie. Znaczenie poszczególnych kodów możesz znaleźć [tutaj](https://pl.wikipedia.org/wiki/Kod_odpowiedzi_HTTP)

Podpowiedź: jeśli przekraczasz dozwolony limit requestów, zatrzymaj kod na określoną liczbę (mili)sekund za pomocą funkcji `time.sleep()`



Czas: ok. 60 minut. Jeśli zostanie Ci czas, spróbuj znaleźć kolejne zastosowanie tego API, odkryj jakie daje możliwości.

In [ ]:
# ...